## The Challenge

## The Solution

1) Create your instrument universe using a range of identifiers

2) Set up a scope to hold the accounts of a subfirm

3) Create a portfolio for each account that exists with the subfirm

4) Create a property to hold the investor and the household

5) Set the initial holdings of the portfolio

5) Create a portfolio group for each household

6) Load market data prices

7) Conduct a valuation against each household

In [10]:
# Import LUSID
import lusid.models as models
import lusid
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import numpy as np
import json
import uuid

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.5.2676.0


### Create your instrument universe using a range of identifiers

In [11]:
equity_instruments = pd.read_csv('data/multiplesystems-instruments-equities.csv')
equity_instruments.head()

,InstrumentName,ClientInternal,Currency,Isin,Figi,ExchangeCode,CountryIssue,Ticker,MarketSector,SecurityType,Coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,LN,united_kingdom,BP/,equity,common_stock,NaN
3,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,LN,united_kingdom,BUR,equity,common_stock,NaN
4,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,LN,united_kingdom,EKF,equity,common_stock,NaN


### Set up a scope to hold the accounts of a subfirm

In [12]:
scope_id = import_data.create_scope_id()
scope = 'subfirm-{}'.format(scope_id)
prettyprint.heading('Scope', scope)

Scope: subfirm-3769-dd6f-6de9-12


### Create a portfolio for each account that exists with the subfirm

In [13]:
accounts = pd.read_csv('data/households-accounts.csv')
accounts.head(n=10)

,account_code,account_name,currency,description,primary_acount_owner_id,household_id,other_account_owner_id
0,040004-929987648,OneIntlBrokerage,USD,Primary trading account,invstr_7325jhv93,hhd_jxgru45055,NaN
1,040004-778939522,OneIntlBrokerage,USD,College fund,invstr_7325jhv93,hhd_jxgru45055,invstr_bbgj93921
2,204507-250099511,VangETF,USD,Passive exchange traded fund,invstr_a1kf37761,hhd_ab3452342,NaN
3,774001-899930233,FidelityOnlineUSEquityOptions,USD,Online US equity & option trades,invstr_bbgj93921,hhd_jxgru45055,NaN
4,400250-613060229,ActiveManagedBlckrock,USD,Actively managed high return investment account,invstr_a1kf37761,hhd_ab3452342,NaN
5,769231-448679901,MutualFund,USD,Pooled investment vehicle,invstr_7325jhv93,hhd_ab3452342,NaN


In [14]:
for index, account in accounts.iterrows():

    # Set the creation date of your portfolio 
    portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)

    # Build your request to create your portfolio
    request = models.CreateTransactionPortfolioRequest(
        display_name=account['account_name'],
        code=account['account_code'],
        base_currency=account['currency'],
        description=account['description'],
        created=portfolio_creation_date,
        corporate_action_source_id=None,
        accounting_method='AverageCost',
        sub_holding_keys=None,
        properties=None)

    # Call LUSID to create your portfolio
    response = client.transaction_portfolios.create_portfolio(
        scope=scope,
        create_request=request)

    # Pretty print the response
    prettyprint.portfolio_response(response)

Portfolio Created
Scope: subfirm-3769-dd6f-6de9-12
Code: 040004-929987648
Portfolio Effective From: 2016-07-18 16:32:33.805599+00:00
Portfolio Created On: 2019-06-05 16:32:34.072178+00:00

Portfolio Created
Scope: subfirm-3769-dd6f-6de9-12
Code: 040004-778939522
Portfolio Effective From: 2016-07-18 16:32:34.930291+00:00
Portfolio Created On: 2019-06-05 16:32:35.195359+00:00

Portfolio Created
Scope: subfirm-3769-dd6f-6de9-12
Code: 204507-250099511
Portfolio Effective From: 2016-07-18 16:32:35.424192+00:00
Portfolio Created On: 2019-06-05 16:32:35.780821+00:00

Portfolio Created
Scope: subfirm-3769-dd6f-6de9-12
Code: 774001-899930233
Portfolio Effective From: 2016-07-18 16:32:36.034693+00:00
Portfolio Created On: 2019-06-05 16:32:36.368708+00:00

Portfolio Created
Scope: subfirm-3769-dd6f-6de9-12
Code: 400250-613060229
Portfolio Effective From: 2016-07-18 16:32:36.516110+00:00
Portfolio Created On: 2019-06-05 16:32:36.777379+00:00

Portfolio Created
Scope: subfirm-3769-dd6f-6de9-12
Code

### Create a property to hold the investor and the household

In [15]:
property_codes = ['household_id', 'primary_acount_owner_id', 'other_account_owner_id']

for property_code in property_codes:

    # Create our request to define a new property
    request = models.CreatePropertyDefinitionRequest(
        domain='Portfolio',
        scope=scope,
        code=property_code,
        value_required=False,
        display_name=property_code,
        data_type_id=models.ResourceId(scope='default', code='string'))

    # Call LUSID to create our new property
    response = client.property_definitions.create_property_definition(
        definition=request)

    # Grab the key off the response to use when referencing this property in other LUSID calls
    portfolio_property_key = response.key

    # Pretty print our key
    prettyprint.heading('Portfolio Property Key', portfolio_property_key)

Portfolio Property Key: Portfolio/subfirm-3769-dd6f-6de9-12/household_id
Portfolio Property Key: Portfolio/subfirm-3769-dd6f-6de9-12/primary_acount_owner_id
Portfolio Property Key: Portfolio/subfirm-3769-dd6f-6de9-12/other_account_owner_id


In [22]:
for index, account in accounts.iterrows():
    response = client.portfolios.upsert_portfolio_properties(
        scope=scope,
        code=account['account_code'],
        portfolio_properties={
            'Portfolio/{}/{}'.format(scope, account_property): models.PropertyValue(
                label_value=account[account_property]
            ) for account_property in property_codes
        }
    )

    # Pretty print the response 
    prettyprint.portfolio_properties_response(response)
    print ('\n')

Properties Sucessfully Updated for Portfolio
Scope:  subfirm-3769-dd6f-6de9-12
Code:  040004-929987648 

Property key: Portfolio/subfirm-3769-dd6f-6de9-12/other_account_owner_id
Value: NaN

Property key: Portfolio/subfirm-3769-dd6f-6de9-12/household_id
Value: hhd_jxgru45055

Property key: Portfolio/subfirm-3769-dd6f-6de9-12/primary_acount_owner_id
Value: invstr_7325jhv93



Properties Sucessfully Updated for Portfolio
Scope:  subfirm-3769-dd6f-6de9-12
Code:  040004-778939522 

Property key: Portfolio/subfirm-3769-dd6f-6de9-12/other_account_owner_id
Value: invstr_bbgj93921

Property key: Portfolio/subfirm-3769-dd6f-6de9-12/household_id
Value: hhd_jxgru45055

Property key: Portfolio/subfirm-3769-dd6f-6de9-12/primary_acount_owner_id
Value: invstr_7325jhv93



Properties Sucessfully Updated for Portfolio
Scope:  subfirm-3769-dd6f-6de9-12
Code:  204507-250099511 

Property key: Portfolio/subfirm-3769-dd6f-6de9-12/other_account_owner_id
Value: NaN

Property key: Portfolio/subfirm-3769-dd6f-6

### Set the initial holdings of the portfolio

### Create a portfolio group for each household

In [23]:
for household in accounts['household_id'].unique():
    
    # Search for portfolios in this household
    # Add them to the portfolio group

hhd_jxgru45055
hhd_ab3452342


### Load market data prices

### Conduct a valuation against each household